In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
complaints_df=pd.read_excel('/content/drive/MyDrive/Team 3 - Complaints.xlsx')

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re

# Load the dataset
#complaints_df = pd.read_excel('/content/drive/MyDrive/Team 3 - Complaints.xlsx')

# Preprocess the narratives
def preprocess_narrative(narrative):
    if pd.isna(narrative):
        return ''
    # Remove non-alphabetic characters and convert to lowercase
    clean_narrative = re.sub(r'[^a-zA-Z\s]', '', str(narrative).lower())
    return clean_narrative

complaints_df['processed_narrative'] = complaints_df['narrative'].apply(preprocess_narrative)



In [11]:
# Split data into features (processed narratives) and target labels
X = complaints_df['processed_narrative']
y = complaints_df['product']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [13]:
# Initialize and train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [14]:
# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.8705
Classification Report:
                     precision    recall  f1-score   support

        credit_card       0.80      0.79      0.79      3212
   credit_reporting       0.90      0.93      0.92     18130
    debt_collection       0.80      0.72      0.76      4619
mortgages_and_loans       0.85      0.82      0.84      3738
     retail_banking       0.87      0.87      0.87      2786

           accuracy                           0.87     32485
          macro avg       0.84      0.83      0.83     32485
       weighted avg       0.87      0.87      0.87     32485



In [15]:
def predict_class(text):
    # Preprocess the input text
    clean_text = preprocess_narrative(text)

    text_tfidf = tfidf_vectorizer.transform([clean_text])
    # Predict the class
    predicted_class = model.predict(text_tfidf)[0]

    if predicted_class in set(y):
        return predicted_class
    else:
        return "Miscellaneous"

In [ ]:
user_input = input("Enter your complaint: ")
predicted_class = predict_class(user_input)
print("Predicted class:", predicted_class)